In [72]:
import cv2
import numpy as np
import mediapipe as mp
import os

In [60]:
# Function to calculate the angle between three points
def calculate_angle(a, b, c):
    a = np.array(a)  # First point
    b = np.array(b)  # Mid point
    c = np.array(c)  # End point
    
    radians = np.arctan2(c[1] - b[1], c[0] - b[0]) - np.arctan2(a[1] - b[1], a[0] - b[0])
    angle = np.abs(radians * 180.0 / np.pi)
    
    if angle > 180.0:
        angle = 360.0 - angle
        
    return angle

In [61]:
def run_model(video_path):

    # Initialize video capture
    cap = cv2.VideoCapture(video_path)

    # Check if video opened successfully
    if not cap.isOpened():
        print(f"Error: Could not open video file {video_path}")
        exit()

    # Curl counter variables
    left_counter = 0
    right_counter = 0
    left_stage = None
    right_stage = None

    # Setup mediapipe instance
    mp_drawing = mp.solutions.drawing_utils
    mp_pose = mp.solutions.pose

    with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
        while cap.isOpened():
            ret, frame = cap.read()
            if not ret:
                print("Error: Could not read frame.")
                break
            
            # Recolor image to RGB
            image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            image.flags.writeable = False
            
            # Make detection
            results = pose.process(image)
            
            # Recolor back to BGR
            image.flags.writeable = True
            image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
            
            # Extract landmarks
            try:
                landmarks = results.pose_landmarks.landmark
                
                # Get coordinates for left arm
                left_shoulder = [landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x,
                                landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y]
                left_elbow = [landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].x,
                            landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].y]
                left_wrist = [landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].x,
                            landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].y]
                
                # Get coordinates for right arm
                right_shoulder = [landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].x,
                                landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].y]
                right_elbow = [landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].x,
                            landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].y]
                right_wrist = [landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].x,
                            landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].y]
                
                # Calculate angle for left arm
                left_angle = calculate_angle(left_shoulder, left_elbow, left_wrist)
                
                # Calculate angle for right arm
                right_angle = calculate_angle(right_shoulder, right_elbow, right_wrist)
                
                # Visualize angle for left arm
                cv2.putText(image, str(left_angle), 
                            tuple(np.multiply(left_elbow, [640, 480]).astype(int)), 
                            cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2, cv2.LINE_AA)
                
                # Visualize angle for right arm
                cv2.putText(image, str(right_angle), 
                            tuple(np.multiply(right_elbow, [640, 480]).astype(int)), 
                            cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2, cv2.LINE_AA)
                
                # Curl counter logic for left arm
                if left_angle > 160:
                    left_stage = "down"
                if left_angle < 30 and left_stage == 'down':
                    left_stage = "up"
                    left_counter += 1
                    print("Left Counter: ", left_counter)
                
                # Curl counter logic for right arm
                if right_angle > 160:
                    right_stage = "down"
                if right_angle < 30 and right_stage == 'down':
                    right_stage = "up"
                    right_counter += 1
                    print("Right Counter: ", right_counter)
            
            except:
                pass
            
            # Render curl counter for left arm
            cv2.rectangle(image, (0, 0), (225, 73), (245, 117, 16), -1)
            
            # Rep data for left arm
            cv2.putText(image, 'LEFT REPS', (15, 12), 
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 1, cv2.LINE_AA)
            cv2.putText(image, str(left_counter), 
                        (10, 60), 
                        cv2.FONT_HERSHEY_SIMPLEX, 2, (255, 255, 255), 2, cv2.LINE_AA)
            
            # Stage data for left arm
            cv2.putText(image, 'STAGE', (120, 12), 
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 1, cv2.LINE_AA)
            cv2.putText(image, left_stage, 
                        (60, 60), 
                        cv2.FONT_HERSHEY_SIMPLEX, 2, (255, 255, 255), 2, cv2.LINE_AA)
            
            # Render curl counter for right arm
            cv2.rectangle(image, (250, 0), (475, 73), (245, 117, 16), -1)
            
            # Rep data for right arm
            cv2.putText(image, 'RIGHT REPS', (265, 12), 
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 1, cv2.LINE_AA)
            cv2.putText(image, str(right_counter), 
                        (260, 60), 
                        cv2.FONT_HERSHEY_SIMPLEX, 2, (255, 255, 255), 2, cv2.LINE_AA)
            
            # Stage data for right arm
            cv2.putText(image, 'STAGE', (385, 12), 
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 1, cv2.LINE_AA)
            cv2.putText(image, right_stage, 
                        (310, 60), 
                        cv2.FONT_HERSHEY_SIMPLEX, 2, (255, 255, 255), 2, cv2.LINE_AA)
            
            # Render detections
            mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                                    mp_drawing.DrawingSpec(color=(245, 117, 66), thickness=2, circle_radius=2), 
                                    mp_drawing.DrawingSpec(color=(245, 66, 230), thickness=2, circle_radius=2))
            
            cv2.imshow('Mediapipe Feed', image)
            
            if cv2.waitKey(10) & 0xFF == ord('q'):
                break

    cap.release()
    cv2.destroyAllWindows()

    return {
        'left': left_counter,
        'right': right_counter,
    }

In [62]:
def get_labelled_videos(folder_name):

    result = []
    
    for video_file_name in os.listdir(os.path.join(os.getcwd(), folder_name)):
        bc = [int(x) for x in video_file_name.split('.')[-2].split('_')[-1].split(',')]
        result.append({
            'filename': video_file_name,
            'actual': {
                'left': bc[0],
                'right': bc[1],
            }
        })

    return result

In [63]:
def error(actual, predicted):
    return {
        'left': predicted['left'] - actual['left'],
        'right': predicted['right'] - actual['right'],
    }

In [64]:
def main():
    all_results = []

    counter = 0
    for labelled_video in get_labelled_videos('videos'):
        counter += 1
        predicted = run_model(
            os.path.join(
                os.getcwd(),
                'videos/' + labelled_video['filename']
            )
        )
        err = error(
            labelled_video['actual'],
            predicted
        )

        all_results.append({
            'filename': labelled_video['filename'],
            'actual': labelled_video['actual'],
            'predicted': predicted,
            'error': err,
        })
        
        print('iteration =', counter, ": error =", err)

    return all_results

In [65]:
results = main()

Left Counter:  1
Left Counter:  2
Left Counter:  3
Right Counter:  1
Right Counter:  2
Left Counter:  4
Left Counter:  5
Right Counter:  3
Right Counter:  4
Left Counter:  6
Right Counter:  5
Left Counter:  7
Right Counter:  6
Left Counter:  8
Right Counter:  7
Error: Could not read frame.
iteration = 1 : error = {'left': 0, 'right': -1}
Left Counter:  1
Left Counter:  2
Left Counter:  3
Error: Could not read frame.
iteration = 2 : error = {'left': 0, 'right': -3}
Right Counter:  1
Left Counter:  1
Right Counter:  2
Left Counter:  2
Right Counter:  3
Left Counter:  3
Right Counter:  4
Left Counter:  4
Error: Could not read frame.
iteration = 3 : error = {'left': 0, 'right': 0}


In [66]:
results

[{'filename': 'b2_8,8.mp4',
  'actual': {'left': 8, 'right': 8},
  'predicted': {'left': 8, 'right': 7},
  'error': {'left': 0, 'right': -1}},
 {'filename': 'b3_3,3.mp4',
  'actual': {'left': 3, 'right': 3},
  'predicted': {'left': 3, 'right': 0},
  'error': {'left': 0, 'right': -3}},
 {'filename': 'H5_4,4.MOV',
  'actual': {'left': 4, 'right': 4},
  'predicted': {'left': 4, 'right': 4},
  'error': {'left': 0, 'right': 0}}]

In [70]:
actual_values = []
predicted_values = []

for result in results:
    actual_values.append(result['actual']['left'])
    actual_values.append(result['actual']['right'])
    predicted_values.append(result['predicted']['left'])
    predicted_values.append(result['predicted']['right'])

# Calculate accuracy manually
correct_predictions = sum(1 for actual, predicted in zip(actual_values, predicted_values) if actual == predicted)
total_predictions = len(actual_values)
accuracy = (correct_predictions / total_predictions) * 100

# Calculate precision, recall, and F1 score manually
true_positives = correct_predictions
false_positives = sum(1 for actual, predicted in zip(actual_values, predicted_values) if actual != predicted and predicted in actual_values)
false_negatives = sum(1 for actual, predicted in zip(actual_values, predicted_values) if actual != predicted and actual in predicted_values)

# Avoid division by zero
precision = true_positives / (true_positives + false_positives) if (true_positives + false_positives) > 0 else 0
recall = true_positives / (true_positives + false_negatives) if (true_positives + false_negatives) > 0 else 0
f1_score = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0

print(f"Accuracy: {accuracy:.2f}%")
print(f"Precision: {precision:.2f}")
print(f"Recall: {recall:.2f}")
print(f"F1 Score: {f1_score:.2f}")

Accuracy: 66.67%
Precision: 1.00
Recall: 0.67
F1 Score: 0.80
